In [1]:
%cd /home/stefano/dev/active/spreads-arb
%load_ext line_profiler

import itertools
import logging
import os
import sys
import pickle
from datetime import date, datetime, timedelta
import shutil

import cryptomart as cm
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import pyutil
import requests
import asyncio
from tardis_client import TardisClient, Channel
from tardis_dev import datasets, get_exchange_details
import vectorbt as vbt
from functools import cached_property

import app
import nest_asyncio
nest_asyncio.apply()
cm_client = cm.Client(quiet=True)

/home/stefano/dev/active/spreads-arb


In [2]:
class TardisData:
    exchange_map = pd.DataFrame(
        [
            {
                "binance": "binance-futures",
                "bitmex": "bitmex",
                "bybit": "bybit",
                "gateio": "gate-io-futures",
                "okex": "okex-swap",
            }
        ]
    ).melt(var_name="cryptomart_exchange", value_name="id")

    def __init__(self):
        self.api_key = os.getenv("TARDIS_API_KEY")
        self.base_url = "https://api.tardis.dev/v1"
        self.data_root_path = os.path.join(os.getenv("ACTIVE_DEV_PATH"), "spreads-arb", "data")
        self.tardis_data_root_path = os.path.join(self.data_root_path, "tardis")
        self.loop = asyncio.get_event_loop()
        self.tardis_exchanges = self.exchange_map.set_index("cryptomart_exchange").iloc[:, 0].to_dict()
        self.cryptomart_exchanges = self.exchange_map.set_index("id").iloc[:, 0].to_dict()

        self.exchange_info = self.load_exchange_info()
        self.instrument_info = self.load_instrument_info()

        self.all_symbols = pd.concat(
            [
                self.get_symbols(exchange, with_spread=False).assign(exchange=exchange)
                for exchange in self.tardis_exchanges
            ]
        )
        self.all_symbols_with_spread = (
            self.all_symbols.join(self.all_symbols, lsuffix="_x", rsuffix="_y", how="cross")
            .pipe(lambda df: df[(df.exchange_x < df.exchange_y) & (df.cryptomart_symbol_x == df.cryptomart_symbol_y)])[
                ["id_x", "cryptomart_symbol_x"]
            ]
            .rename(columns={"id_x": "id", "cryptomart_symbol_x": "cryptomart_symbol"})[["cryptomart_symbol"]]
            .drop_duplicates()
        )
        logging.getLogger("tardis_dev.datasets.download").addHandler(logging.StreamHandler(sys.stdout))

    def load_exchange_info(self):
        exchange_info_futures = [get_exchange_details(exchange) for exchange in self.tardis_exchanges.values()]
        exchange_info = self.loop.run_until_complete(asyncio.gather(*exchange_info_futures))
        return {self.cryptomart_exchanges[info["id"]]: info for info in exchange_info}

    def load_instrument_info(self):
        return {
            exchange: cm_client.instrument_info(exchange, "perpetual", cache_kwargs={"refresh": False})
            for exchange in self.cryptomart_exchanges.values()
        }

    @staticmethod
    def get_data_filename(exchange, data_type, date, symbol, format):
        return f"{exchange}/{data_type}/{symbol}/{date.strftime('%Y-%m-%d')}.{format}.gz"

    def get_symbol_id(self, exchange, cryptomart_symbol, ignore_errors=False):
        if not isinstance(cryptomart_symbol, list):
            cryptomart_symbol = [cryptomart_symbol]
            ret_fn = lambda x: x[0]
        else:
            ret_fn = lambda x: list(x)

        cryptomart_symbols = pd.Series(cryptomart_symbol, name="cryptomart_symbol")

        symbols = self.all_symbols[self.all_symbols.exchange == exchange].merge(cryptomart_symbols, how="right")
        if not ignore_errors and symbols.id.isna().any():
            raise ValueError(f"{list(symbols[symbols.id.isna()].cryptomart_symbol)} not found on exchange {exchange}")

        return ret_fn(symbols.id)

    def get_symbols(
        self,
        exchange,
        data_types=["quotes", "derivative_ticker"],
        from_date="2023-04-10",
        to_date="2023-05-10",
        with_spread=True,
    ):
        all_symbols = pd.DataFrame(self.exchange_info[exchange]["datasets"]["symbols"]).pipe(
            lambda df: df[
                (df.type == "perpetual")
                & (df.availableSince <= from_date)
                & (df.availableTo >= to_date)
                & (df.dataTypes.apply(lambda l: np.isin(data_types, l).all()))
            ]
        )[["id"]]
        cryptomart_symbols = self.instrument_info[exchange][["cryptomart_symbol", "exchange_symbol"]].rename(
            columns={"exchange_symbol": "id"}
        )
        if not with_spread:
            return all_symbols.merge(cryptomart_symbols)
        else:
            return all_symbols.merge(cryptomart_symbols).merge(self.all_symbols_with_spread)

    def download(
        self,
        exchange,
        data_types=["trades"],
        from_date="2023-01-12",
        to_date="2023-05-04",
        symbols=None,
        compress=False,
        **kwargs,
    ):
        if symbols is None:
            symbols = self.get_symbols(exchange, data_types, from_date, to_date, with_spread=True)
        else:
            all_symbols = self.get_symbols(exchange, data_types, from_date, to_date, with_spread=False)
            symbols = all_symbols.merge(pd.Series(symbols, name="cryptomart_symbol"), how="right")
            unavailable_symbols = symbols[symbols.id.isna()].cryptomart_symbol.unique()
            if len(unavailable_symbols) > 0:
                print("Warning: some symbols are not available for download", unavailable_symbols)
            symbols = symbols.dropna()

        tardis_exchange = self.tardis_exchanges[exchange]

        future = datasets.download(
            exchange=tardis_exchange,
            data_types=data_types,
            from_date=from_date,
            to_date=to_date,
            symbols=list(symbols.id),
            api_key=self.api_key,
            download_dir=self.tardis_data_root_path,
            get_filename=self.get_data_filename,
            **kwargs,
        )

        self.loop.run_until_complete(future)

        if compress:
            for data_type in data_types:
                if data_type not in ["derivative_ticker", "quotes"]:
                    print("No compression implemented for data type", data_type)
                    continue
                for symbol in symbols.cryptomart_symbol:
                    if data_type == "derivative_ticker":
                        self.load_ticker(exchange, symbol, from_date, to_date, compress=True)
                    elif data_type == "quotes":
                        self.load_quotes(exchange, symbol, from_date, to_date, compress=True)

    def data_iterator(self, exchange, symbol, data_type, from_date="2023-01-12", to_date="2023-05-04"):
        start_time = datetime.fromisoformat(from_date)
        end_time = datetime.fromisoformat(to_date)
        day_timedelta = timedelta(days=1)
        tardis_symbol = self.get_symbol_id(exchange, symbol)
        tardis_exchange = self.tardis_exchanges[exchange]

        for day in range((end_time - start_time).days):
            date = start_time + day * day_timedelta
            yield os.path.join(
                self.tardis_data_root_path,
                self.get_data_filename(tardis_exchange, data_type, date, tardis_symbol, "csv"),
            )

    def load_trades(self, exchange, symbol, from_date="2023-01-12", to_date="2023-05-04"):
        dfs = []
        for filename in self.data_iterator(exchange, symbol, "trades", from_date, to_date):
            dfs.append(pd.read_csv(filename))
        return pd.concat(dfs, ignore_index=True)

    def load_ticker(self, exchange, symbol, from_date="2023-01-12", to_date="2023-05-04", compress=False):
        dfs = []
        for filename in self.data_iterator(exchange, symbol, "derivative_ticker", from_date, to_date):
            dfs.append(pd.read_csv(filename, usecols=["timestamp", "last_price"]))
        df = pd.concat(dfs, ignore_index=True)
        df["timestamp"] = df.timestamp.apply(lambda x: datetime.utcfromtimestamp(x / 1e6))
        df = df.dropna()

        if compress:
            outpath = os.path.join(self.data_root_path, "tick_prices", exchange, f"{symbol}.parquet")
            os.makedirs(os.path.dirname(outpath), exist_ok=True)
            df.to_parquet(outpath)
            shutil.rmtree(os.path.dirname(filename))

        return df

    def load_quotes(self, exchange, symbol, from_date="2023-01-12", to_date="2023-05-04", compress=False):
        dfs = []
        for filename in self.data_iterator(exchange, symbol, "quotes", from_date, to_date):
            dfs.append(
                pd.read_csv(filename, usecols=["timestamp", "ask_price", "ask_amount", "bid_price", "bid_amount"])
            )
        df = pd.concat(dfs, ignore_index=True)
        df["timestamp"] = df.timestamp.apply(lambda x: datetime.utcfromtimestamp(x / 1e6))
        df = df.dropna()
        # def largest_spread(g):
        #     g["bas"] = round(g["ask_price"] - g["bid_price"], 15)
        #     return g.groupby("bas", as_index=True)[["bid_amount", "ask_amount"]].sum().iloc[:3]

        # df = df.resample(timedelta(hours=1), on="timestamp").apply(largest_spread).reset_index()

        if compress:
            outpath = os.path.join(self.data_root_path, "tick_quotes", exchange, f"{symbol}.parquet")
            os.makedirs(os.path.dirname(outpath), exist_ok=True)
            df.to_parquet(outpath)
            shutil.rmtree(os.path.dirname(filename))

        return df


td = TardisData()

# ohlcvs = app.data_prep.all_ohlcv(
#     "2022-02-01", "2023-05-04", "interval_1h", refresh=False, identifiers=["spreads-arb-v2"]
# )
# ohlcvs = ohlcvs[ohlcvs.missing_rows <= 0]

# for exchange in ohlcvs.index.levels[0]:
#     try:
#         symbols = ohlcvs.loc[exchange].reset_index().symbol.unique()
#     except KeyError:
#         continue
#     # td.download(exchange, ["quotes"], "2023-02-20", "2023-05-04", symbols, concurrency=10)

2023-05-20 18:48:09.526 INFO     pyutil.cache                                                                      log:89   Using cached value in call to instrument_info((id:binance_instrument_info_perpetual), map_column=None) | key=95f609d61b5ba5e3d768cb5fe6b0184a (/tmp/cache/instrument_info/binance/perpetual)...
2023-05-20 18:48:09.532 INFO     pyutil.cache                                                                      log:89   Using cached value in call to instrument_info((id:bitmex_instrument_info_perpetual), map_column=None) | key=6ac81cd55e6a7b38b91de750b333d130 (/tmp/cache/instrument_info/bitmex/perpetual)...
2023-05-20 18:48:09.535 INFO     pyutil.cache                                                                      log:89   Using cached value in call to instrument_info((id:bybit_instrument_info_perpetual), map_column=None) | key=8901251cd26097a04a26eb972bcfd1ee (/tmp/cache/instrument_info/bybit/perpetual)...
2023-05-20 18:48:09.539 INFO     pyutil.cache             

In [8]:
x.memory_usage() / 1e6

Index            0.000128
timestamp     1002.694592
ask_amount    1002.694592
ask_price     1002.694592
bid_price     1002.694592
bid_amount    1002.694592
dtype: float64

In [20]:
x.resample("3s", on="timestamp").mean().memory_usage() / 1e6

Index         6.912
ask_amount    6.912
ask_price     6.912
bid_price     6.912
bid_amount    6.912
dtype: float64

In [86]:
y = pd.read_parquet("/home/stefano/dev/active/spreads-arb/data/tick_quotes/okex/1INCH.parquet")

In [89]:
y.set_index("timestamp").interpolate(method="time").resample("1s", on="timestamp").mean()

,ask_amount,ask_price,bid_price,bid_amount
timestamp,,,,
2023-04-10 00:00:00,184.0,0.5333,0.5332,1070.5
2023-04-10 00:00:01,184.0,0.5333,0.5332,2238.5
2023-04-10 00:00:02,NaN,NaN,NaN,NaN
2023-04-10 00:00:03,184.0,0.5333,0.5332,2333.5
2023-04-10 00:00:04,514.0,0.5333,0.5332,3157.8
...,...,...,...,...
2023-05-09 23:59:52,664.0,0.4155,0.4154,836.0
2023-05-09 23:59:53,364.0,0.4155,0.4154,831.0
2023-05-09 23:59:54,NaN,NaN,NaN,NaN


In [11]:
y.memory_usage() / 1e6

Index          0.000128
timestamp     59.753576
ask_amount    59.753576
ask_price     59.753576
bid_price     59.753576
bid_amount    59.753576
dtype: float64

In [3]:
td.download("binance", ["quotes"], "2023-04-10", "2023-05-10", compress=False)

In [4]:
td.download("bitmex", ["quotes"], "2023-04-10", "2023-05-10", compress=False)
td.download("okex", ["quotes"], "2023-04-10", "2023-05-10", compress=False)
td.download("bybit", ["quotes"], "2023-04-10", "2023-05-10", compress=False)
td.download("gateio", ["quotes"], "2023-04-10", "2023-05-10", compress=False)

KeyboardInterrupt: 

In [35]:
x = td.load_bas("binance", "BTC", "2023-04-10", "2023-05-10")

In [36]:
x

,timestamp,ask_amount,ask_price,bid_price,bid_amount
0,2023-04-10 00:00:00.208,1.640,28309.4,28309.3,7.566
1,2023-04-10 00:00:03.848,2.345,28309.4,28309.3,7.566
2,2023-04-10 00:00:03.957,1.369,28309.4,28309.3,7.566
3,2023-04-10 00:00:04.037,1.351,28309.4,28309.3,7.566
4,2023-04-10 00:00:04.139,1.350,28309.4,28309.3,7.566
...,...,...,...,...,...
59811868,2023-05-09 23:59:59.896,17.122,27610.3,27610.2,3.087
59811869,2023-05-09 23:59:59.933,17.122,27610.3,27610.2,3.067
59811870,2023-05-09 23:59:59.957,17.122,27610.3,27610.2,3.063
59811871,2023-05-09 23:59:59.976,17.122,27610.3,27610.2,2.963


In [37]:
y = pd.read_parquet("/home/stefano/dev/active/spreads-arb/data/tardis/binance-futures/derivative_ticker/BTCUSDT.parquet")

In [38]:
y

,timestamp,last_price
1,2023-04-10 00:00:00.211,28309.4
2,2023-04-10 00:00:01.004,28309.4
3,2023-04-10 00:00:02.000,28309.4
4,2023-04-10 00:00:02.000,28309.4
5,2023-04-10 00:00:04.006,28309.4
...,...,...
4988546,2023-05-09 23:59:56.013,27610.9
4988547,2023-05-09 23:59:56.937,27610.7
4988548,2023-05-09 23:59:58.000,27610.7
4988549,2023-05-09 23:59:58.941,27610.2


In [4]:
td.download("binance", ["derivative_ticker"], "2023-04-10", "2023-05-10", ["BTC"])

In [18]:
x = td.load_ticker("binance", "BTC", "2023-04-10", "2023-05-10", compress=True)

In [14]:
x.to_parquet("/home/stefano/dev/active/spreads-arb/data/tardis/binance-futures/derivative_ticker/BTCUSDT/BTCUSDT.parquet")

In [15]:
pd.read_parquet("/home/stefano/dev/active/spreads-arb/data/tardis/binance-futures/derivative_ticker/BTCUSDT/BTCUSDT.parquet")

,timestamp,last_price
1,2023-04-10 00:00:00.211,28309.4
2,2023-04-10 00:00:01.004,28309.4
3,2023-04-10 00:00:02.000,28309.4
4,2023-04-10 00:00:02.000,28309.4
5,2023-04-10 00:00:04.006,28309.4
...,...,...
4988546,2023-05-09 23:59:56.013,27610.9
4988547,2023-05-09 23:59:56.937,27610.7
4988548,2023-05-09 23:59:58.000,27610.7
4988549,2023-05-09 23:59:58.941,27610.2


In [30]:
for exchange in ohlcvs.index.levels[0]:
    try:
        symbols = ohlcvs.loc[exchange].reset_index().symbol.unique()
    except:
        continue
    for symbol in symbols:
        try:
            tardis_symbol = td.exchange_datasets(exchange).set_index("cryptomart_symbol").loc[symbol, "id"]
            assert len(os.listdir(os.path.join(td.tardis_data_root_path, td.tardis_exchanges[exchange], "quotes", tardis_symbol))) == 73
        except:
            continue
        
        ohlcvs.at[(exchange, "perpetual", symbol), "has_bas"] = True
        
        
display(ohlcvs[ohlcvs.has_bas == True])
display(ohlcvs[ohlcvs.has_bas != True])

ohlcv  \
exchange inst_type symbol                                                      
bitmex   perpetual DOGE    b'\x80\x04\x95\xcc\x01\x00\x00\x00\x00\x00\x00...   
                   DOT     b'\x80\x04\x95\xcc\x01\x00\x00\x00\x00\x00\x00...   
                   ADA     b'\x80\x04\x95\xcc\x01\x00\x00\x00\x00\x00\x00...   
                   BNB     b'\x80\x04\x95\xcc\x01\x00\x00\x00\x00\x00\x00...   
                   SOL     b'\x80\x04\x95\xcc\x01\x00\x00\x00\x00\x00\x00...   
...                                                                      ...   
binance  perpetual ENS     b'\x80\x04\x95\x12\x01\x00\x00\x00\x00\x00\x00...   
                   PEOPLE  b'\x80\x04\x95\x12\x01\x00\x00\x00\x00\x00\x00...   
                   ANT     b'\x80\x04\x95\x12\x01\x00\x00\x00\x00\x00\x00...   
                   ROSE    b'\x80\x04\x95\x12\x01\x00\x00\x00\x00\x00\x00...   
                   DUSK    b'\x80\x04\x95\x12\x01\x00\x00\x00\x00\x00\x00...   

                              rows  missing_rows earliest_time  \
exchange inst_type symbol                                        
bitmex   perpetual DOGE    10968.0           0.0    2022-02-01   
                   DOT     10968.0           0.0    2022-02-01   
                   ADA     10968.0           0.0    2022-02-01   
                   BNB     10968.0           0.0    2022-02-01   
                   SOL     10968.0           0.0    2022-02-01   
...                            ...           ...           ...   
binance  perpetual ENS     10968.0           0.0    2022-02-01   
                   PEOPLE  10968.0           0.0    2022-02-01   
                   ANT     10968.0           0.0    2022-02-01   
                   ROSE    10968.0           0.0    2022-02-01   
                   DUSK    10968.0           0.0    2022-02-01   

                                  latest_time  gaps has_bas  
exchange inst_type symbol                                    
bitmex   perpetual DOGE   2023-05-03 23:00:00   0.0    True  
                   DOT    2023-05-03 23:00:00   0.0    True  
                   ADA    2023-05-03 23:00:00   0.0    True  
                   BNB    2023-05-03 23:00:00   0.0    True  
                   SOL    2023-05-03 23:00:00   0.0    True  
...                                       ...   ...     ...  
binance  perpetual ENS    2023-05-03 23:00:00   0.0    True  
                   PEOPLE 2023-05-03 23:00:00   0.0    True  
                   ANT    2023-05-03 23:00:00   0.0    True  
                   ROSE   2023-05-03 23:00:00   0.0    True  
                   DUSK   2023-05-03 23:00:00   0.0    True  

[488 rows x 7 columns]

,,,ohlcv,rows,missing_rows,earliest_time,latest_time,gaps,has_bas
exchange,inst_type,symbol,,,,,,,
gateio,perpetual,TON,b'\x80\x04\x95\x12\x01\x00\x00\x00\x00\x00\x00...,10968.0,0.0,2022-02-01,2023-05-03 23:00:00,0.0,NaN


In [33]:
params = list(ohlcvs[ohlcvs.has_bas == True].reset_index().apply(lambda r: (r.exchange, r.symbol), axis=1))

In [35]:
from concurrent.futures import ThreadPoolExecutor

def worker_fn(exchange, symbol):
    try:
        tardis_symbol = td.exchange_datasets(exchange).set_index("cryptomart_symbol").at[symbol, "id"]
    except KeyError:
        print(f"Skipping {exchange} {symbol}")
        
    print(f"Processing {exchange} {symbol}")
    bas = td.load_bas(exchange, symbol, "2023-02-20", "2023-05-04")
    
    outpath = os.path.join(
        os.getenv("ACTIVE_DEV_PATH"), "spreads-arb", "data", "bid_ask_spreads_1h", exchange, f"{symbol}.pkl"
    )
    os.makedirs(os.path.dirname(outpath), exist_ok=True)
    bas.to_pickle(outpath)
    
    tardis_exchange = td.tardis_exchanges[exchange]
    shutil.rmtree(os.path.join(td.tardis_data_root_path, tardis_exchange, "quotes", tardis_symbol), ignore_errors=True)
    print(f"Done {exchange} {symbol}")
    

with ThreadPoolExecutor(max_workers=10) as executor:
    executor.map(lambda p: worker_fn(*p), [("gateio", "XVS"),])

Processing gateio XVS


In [14]:
x = pd.read_csv(
    "/home/stefano/dev/active/spreads-arb/data/tardis/binance-futures/quotes/2023-01-12_DOGEUSDT.csv.gz",
    usecols=["timestamp", "ask_price", "ask_amount", "bid_price", "bid_amount"],
)
x["timestamp"] = x.timestamp.apply(lambda x: datetime.utcfromtimestamp(x / 1e6))


def weighted_average(g):
    ask_amount = g["ask_amount"].sum()
    ask_price = np.average(g["ask_price"], weights=g["ask_amount"], keepdims=False)
    bid_amount = g["bid_amount"].sum()
    bid_price = np.average(g["bid_price"], weights=g["bid_amount"], keepdims=False)
    return pd.Series(
        {"ask_amount": ask_amount, "ask_price": ask_price, "bid_amount": bid_amount, "bid_price": bid_price}
    )

def largest_spread(g):
    g["bas"] = round(g["ask_price"] - g["bid_price"], 15)
    return g.groupby("bas", as_index=True)[["bid_amount", "ask_amount"]].sum().iloc[:3]


y = x.resample(timedelta(hours=1), on="timestamp").apply(weighted_average).reset_index()
y["ask_capacity"] = y["ask_amount"] * y["ask_price"]
y["bid_capacity"] = y["bid_amount"] * y["bid_price"]
z = x.resample(timedelta(hours=1), on="timestamp").apply(largest_spread).reset_index()

In [ ]:
def get_bid_ask_spread(exchange, )

In [15]:
z

,timestamp,bas,bid_amount,ask_amount
0,2023-01-12 00:00:00,0.00001,7134927905,6316559567
1,2023-01-12 00:00:00,0.00002,81091826,87455642
2,2023-01-12 00:00:00,0.00003,11287436,11254392
3,2023-01-12 01:00:00,0.00001,4565471075,5470379859
4,2023-01-12 01:00:00,0.00002,46414493,79717705
...,...,...,...,...
64,2023-01-12 22:00:00,0.00002,20564383,19171973
65,2023-01-12 22:00:00,0.00003,1840962,1068824
66,2023-01-12 23:00:00,0.00001,5866132144,6007276513
67,2023-01-12 23:00:00,0.00002,27871690,27699958


In [237]:
td.load_trades("bitmex", "BTC")

2023-05-12 16:21:29.555 INFO     pyutil.cache                                                                      log:89   Using cached value in call to instrument_info((id:bitmex_instrument_info_perpetual), map_column=None) | key=6ac81cd55e6a7b38b91de750b333d130 (/tmp/cache/instrument_info/bitmex/perpetual)...
INFO:pyutil.cache:Using cached value in call to instrument_info((id:bitmex_instrument_info_perpetual), map_column=None) | key=6ac81cd55e6a7b38b91de750b333d130 (/tmp/cache/instrument_info/bitmex/perpetual)...


,exchange,symbol,timestamp,local_timestamp,id,side,price,amount
0,bitmex,XBTUSDT,1673481604346000,1673481604357860,db415454-e477-0401-56e1-36dacd0fb8e0,sell,17944.5,440000
1,bitmex,XBTUSDT,1673481604346000,1673481604357860,f43b8f26-9778-6bbf-a709-a3c84fcfd566,sell,17944.5,225000
2,bitmex,XBTUSDT,1673481604346000,1673481604357860,56658b6e-d067-05c1-d56c-6839135d8a78,sell,17944.5,100000
3,bitmex,XBTUSDT,1673481604346000,1673481604357860,d465fc30-d0e3-5b1b-f72c-f583c2d5ebfe,sell,17944.5,119000
4,bitmex,XBTUSDT,1673481604346000,1673481604357860,f9e4d208-85de-ef4d-7326-016c0eb35f08,sell,17944.5,28000
...,...,...,...,...,...,...,...,...
1367410,bitmex,XBTUSDT,1683158340181000,1683158340193588,72da5386-bc78-25d2-bf5e-f24cd9f6cd25,buy,28988.5,3000
1367411,bitmex,XBTUSDT,1683158346826000,1683158346835007,398d75d3-80d4-fcf4-c98e-a4b22dcf760f,buy,28988.5,37000
1367412,bitmex,XBTUSDT,1683158353188000,1683158353197139,ac585cf3-09e3-60fd-2a81-a7757b176dc0,buy,28994.0,65000
1367413,bitmex,XBTUSDT,1683158353205000,1683158353216152,82590cc9-faa5-afba-b552-30732684ffa7,buy,28997.5,202000
